# Planet Order and Delivery to GEE

*Notebook based on Planetlabs example notebooks for [GEE Delivery](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/gee-integration/gee-integration.ipynb) and the [Data API Introduction for Python](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/planet_data_api_introduction.ipynb)*

- *click [here](https://developers.planet.com/docs/apis/data/) for general info on the Planet API*
- *click [here](https://developers.planet.com/docs/data/psscene/) for more info on the specifications of the PlanetScope Satellite products and the general order process*

**Prerequisites:**
- Planet's Python SDK 2.0 installed and initialized in your environment. Click [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/get-started/quick-start-guide/) for more info and instructions.
- An AOI : `AOI`
- A GEE project with EE API enabled `EE-Project-ID`
- A pre-existing GEE ImageCollection `planet_collection`
- An account with a download quota

## Planet Ordering Set-up

In [ ]:
import json
import os # to access enviornmental modules
import requests # to create http-requests
import pathlib
import time

from planet import Session, DataClient, OrdersClient

# set Planet API key as environment variable (find it under "My Account"->"My Settings"
os.environ['PL_API_KEY']='PLACE_YOUR_KEY_HERE'

PLANET_API_KEY = os.getenv('PL_API_KEY')
# Setup the API Key from the `PL_API_KEY` environment variable

BASE_URL = "https://api.planet.com/data/v1"
ORDERS_URL = 'https://api.planet.com/compute/ops/orders/v2' 

#setup a session
session = requests.Session()

#authenticate session with user name and password, pass in an empty string for the password
session.auth = (PLANET_API_KEY, "")

#make a get request to the Data API
res = session.get(BASE_URL)

print(res.status_code)
# test response

print(res.text)
# print response body

### GEE Set-up

In [ ]:
# Google Earth Engine configuration
# Define cloud delivery location:
delivery_config = {
        "google_earth_engine": {
            "project": 'EE-Project-ID',
            "collection": 'planet_collection'
        }
}

## Definition of AOI
--> needs to be one feature, so no feature collection is allowed! If there are multiple features, these should be merged into one single Multipolygon-object.

Options: Either delineate features manually via https://geojson.io/ or merge together and make sure that the merged object represents a Feature and not a FeatureCollection

In [ ]:
# AOI defined as GeoJSON Multipolygon 

AOI = {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              [
                5.113263255620006,
                44.34292628821391
              ],
              [
                5.115578547198506,
                44.34188534442477
              ],
              [
                5.11424315052414,
                44.34033266130699
              ],
              [
                5.112564678872293,
                44.339201208222406
              ],
              [
                5.111951252847493,
                44.33899518862323
              ],
              [
                5.11139955489029,
                44.33818283698562
              ],
              [
                5.109007502296916,
                44.33736265820315
              ],
              [
                5.108368368299897,
                44.33691269507484
              ],
              [
                5.106925963096304,
                44.3368928987703
              ],
              [
                5.106003174226122,
                44.336692857792336
              ],
              [
                5.10574952447209,
                44.33665265877533
              ],
              [
                5.104430680987621,
                44.335916869103734
              ],
              [
                5.103572963817739,
                44.336727131908276
              ],
              [
                5.106578812029752,
                44.33858562790278
              ],
              [
                5.107329941529888,
                44.33916678470264
              ],
              [
                5.107567466368412,
                44.33953951673553
              ],
              [
                5.10790830734557,
                44.34003199668375
              ],
              [
                5.108556067933393,
                44.34016127076981
              ],
              [
                5.109532576206893,
                44.34070174740563
              ],
              [
                5.109625302362653,
                44.34110144986237
              ],
              [
                5.111011212797722,
                44.34187627242806
              ],
              [
                5.111568185661511,
                44.3423049981494
              ],
              [
                5.112167038427432,
                44.3424296644817
              ],
              [
                5.112825524063508,
                44.3426110841007
              ],
              [
                5.113263255620006,
                44.34292628821391
              ]
            ]
          ]
        ]
      }

## Setting up image filter

In [ ]:
# Daterange filter
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2019-01-01T00:00:00.000Z", # "gte" -> Greater than or equal to
        "lte": "2019-12-31T23:59:59.999Z" # "lte" -> Lower than or equal to
    }
}

# geometry filter on defined AOI
geom_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": AOI
}

# quality assurance
quality_filter = {
        "type": "StringInFilter",
        "field_name": "quality_category",
        "config": ["standard"]
      }

# cloud pre-filtering
cloud_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# permission pre-filtering
permission_filter = {
  "type": "PermissionFilter",
  "config": ["assets:download"]
}

# Setup an "AND" logical filter
image_filter = {
    "type": "AndFilter",
    "config": [date_filter, geom_filter, quality_filter, cloud_filter, permission_filter]
}

# Print the logical filter
# p(and_filter)

## Requesting image IDs fitting the filter

In [ ]:
# Setup the quick search endpoint url
search_url = "{}/quick-search".format(BASE_URL)

# Construct the request.
request = {
    "item_types" : ["PSScene"],
    "filter" : image_filter
}

# Send the POST request to the API quick search endpoint
res = session.post(search_url, json=request)
geojson = res.json()

# Initialize the item_ids list to collect all item ids
item_ids = []

# Collect item IDs from current page
features = geojson["features"]
current_page_item_ids = [f['id'] for f in features]
item_ids.extend(current_page_item_ids)

# loop to paginate through all available pages to extract >250 items
while current_page_item_ids:

    # Check if there's a next page, if not, break the loop
    next_url = geojson["_links"]["_next"] 
    
    # Update the request URL for the next iteration
    res_next = session.get(next_url)
    geojson = res_next.json()
    
    # Collect item IDs from next page
    features_next = geojson["features"]
    current_page_item_ids = [f['id'] for f in features_next]
    item_ids.extend(current_page_item_ids)

# Print the total number of item IDs collected
print(len(item_ids))

# print all Item IDs collected
print(item_ids)

### Filter images - include interval of 5 days for image selection

In [ ]:
from datetime import datetime, timedelta

# items to which the access is not allowed
to_be_removed = ['20190417_130415_0f4c', '20190628_101130_1105', '20190808_101400_1105', '20191204_101748_1105']

# remove them from initial list
image_ids = [item for item in item_ids if item not in to_be_removed]

# filter to only get item with predefined interval
def filter_images_by_interval(image_ids, interval_days):
    # Convert image IDs to datetime objects and sort
    dates_images = sorted([
        (datetime.strptime(image_id.split('_')[0], '%Y%m%d'), image_id)
        for image_id in image_ids
    ], key=lambda x: x[0])

    # Initialize variables
    filtered_image_ids = []
    last_selected_date = None

    for date_image in dates_images:
        date, image_id = date_image

        # If this is the first image or the date is at least interval_days after the last selected date, select it
        if last_selected_date is None or date >= last_selected_date + timedelta(days=interval_days):
            last_selected_date = date
            # Find and include all images for the selected date
            filtered_image_ids.extend([
                img_id for img_date, img_id in dates_images if img_date == date
            ])
            
    return filtered_image_ids

# Call the function
filtered_image_ids = filter_images_by_interval(image_ids, interval_days=5)

# Print number and names of the selected image IDs
print(len(filtered_image_ids), filtered_image_ids)

## Building the order request

In [ ]:
# Product description for the order request, including image_ids from filter
data_products = [
    {
        "item_ids":       filtered_image_ids,
        "item_type":      'PSScene', # analytic surface reflectance 8-band multispectral band analytic_8b_sr_udm2
        "product_bundle": 'analytic_sr_udm2'
    }
]

tools = [
    {
      "clip": {
        "aoi": AOI
      }
    },
    {
        "harmonize": {
            "target_sensor": 'Sentinel-2'
        }
    }
]

# Build the final order request
planet_order = {
    "name":     '2018_eygue4_order',
    "products": data_products,
    "delivery": delivery_config,
    "tools":    tools
}

print(planet_order)

## Placing the Order for Delivery to GEE

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

# place order !
response = requests.post(ORDERS_URL, data=json.dumps(planet_order), auth=session.auth, headers=headers)

# print order infos
response.json()

# INFO: if order is denied due to a denied access to assets, you can exclude these IDs by adding them to the list 'to_be_removed' in the filtering function

### Check order state

In [ ]:
# get link of order
order_link = ORDERS_URL + '/' + response.json()['id']

# print order state
print(requests.get(order_link, auth=session.auth).json()['state'])